In [1]:
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import random
from fake_useragent import UserAgent

import csv
import openpyxl  # Library for creating XLSX files

# AUTO CRAWL

In [19]:
def get_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    ua = UserAgent()
    user_agent = ua.random
    print(user_agent)
    # Adding argument to disable the AutomationControlled flag 
    chrome_options.add_argument("--disable-blink-features=AutomationControlled") 
    # Exclude the collection of enable-automation switches 
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
    # Turn-off userAutomationExtension 
    chrome_options.add_experimental_option("useAutomationExtension", False) 
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--profile-directory=Profile 1")
    user_dir = r'C:/Users/Admin/AppData/Local/Google/Chrome/User Data'
    chrome_options.add_argument(f'--user-data-dir={user_dir}')
    driver = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
    return driver

driver = get_chrome_driver()

# Hàm auto scroll, refresh sau nhiều lần và click next chương bằng Selenium
def open_and_refresh_website(url):
    driver.get(url)
    page_source = driver.page_source
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    time.sleep(3)
    try:
        for _ in range(15):
            next_link = driver.find_element(By.CSS_SELECTOR, "a[href*='#page'][rel='next']")  # Using CSS selector
            if next_link:
                time.sleep(1)
                next_link.click()
                time.sleep(1)
            else:
                break
    except WebDriverException as e:
        print(f"WebDriverException: {e}")
    restaurant_data = []
    restaurant_elements = driver.find_elements(By.CLASS_NAME, "resname")
    print(f"Found {len(restaurant_elements)} restaurants")
    # Iterate through each restaurant element
    for restaurant in restaurant_elements:
        restaurant_name_element = restaurant.find_element(By.TAG_NAME, "h2")
        result_address = restaurant.find_element(By.CLASS_NAME, 'result-address')
        restaurant_name = restaurant_name_element.text
        address = result_address.find_element(By.CLASS_NAME, 'address')
        if(result_address):
            # Extract address text
            dia_chi = result_address.text
            #Quận nằm trong class="address", span thứ 2
            spans = result_address.find_elements(By.CSS_SELECTOR, 'div span')
            if(spans):
                quan = spans[1].text

                
        restaurant_data.append([restaurant_name, dia_chi, quan])
        print("Tên:" + restaurant_name + "\tĐịa chỉ: "+ dia_chi + "\tQuận: " + quan)
    # Create XLSX workbook and worksheet
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.title = "Dữ liệu"
    # Write headers
    sheet.append(["Tên", "Địa Chỉ", "Quận"])
    # Write restaurant data
    for row in restaurant_data:
        sheet.append(row)
    # Save the XLSX file
    wb.save("data_quan_1_2.xlsx")


def crawl_foody(url):
    driver.get(url)
    try:
        for _ in range(15):
            next_link = driver.find_element(By.CSS_SELECTOR, "a[href*='#page'][rel='next']")  # Using CSS selector
            if next_link:
                time.sleep(1)
                next_link.click()
                time.sleep(2)
            else:
                break
    except WebDriverException as e:
        print(f"WebDriverException: {e}")
    time.sleep(2)
    page_source = driver.page_source
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    restaurant_data = []
    restaurant_elements = soup.find_all("div", class_="resname")
    print(f"Found {len(restaurant_elements)} restaurants")
    for restaurant in restaurant_elements:
        # Extract restaurant name (modify selector as needed)
        restaurant_name_element = restaurant.find("h2")  # Modify selector for name if needed
        address = restaurant.find('div', class_='address')
        spans = address.find_all('span')
        
        restaurant_name = restaurant_name_element.text.strip() if restaurant_name_element else ""  # Handle potential absence
        
        dia_chi = ""
        quan = ""
        if(spans): 
            dia_chi = spans[0].get_text()
            quan = spans[1].get_text()
        restaurant_data.append([restaurant_name, dia_chi, quan])
        print("Tên:" + restaurant_name + "\tĐịa chỉ: "+ dia_chi + "\tQuận: " + quan)
    # Create XLSX workbook and worksheet
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.title = "Dữ liệu"
    # Write headers
    sheet.append(["Tên", "Địa Chỉ", "Quận"])
    # Write restaurant data
    for row in restaurant_data:
        sheet.append(row)
    # Save the XLSX file
    wb.save("data_quan_1_2.xlsx")


# Set the website URL
website_urlBNS = "https://www.foody.vn/ho-chi-minh/dia-diem?CategoryGroup=food&q=l%E1%BA%A9u%20b%C3%B2&dtids=1,4"

# Số giây cách
refresh_interval = 3

# Gọi đến function auto refresh và auto click next chương
crawl_foody(website_urlBNS)

Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36
WebDriverException: Message: element not interactable
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00D3B8E3+45827]
	(No symbol) [0x00CCDCC4]
	(No symbol) [0x00BC138D]
	(No symbol) [0x00C03074]
	(No symbol) [0x00BF85EF]
	(No symbol) [0x00C22DFC]
	(No symbol) [0x00BF8075]
	(No symbol) [0x00C23094]
	(No symbol) [0x00C3C034]
	(No symbol) [0x00C22B96]
	(No symbol) [0x00BF6998]
	(No symbol) [0x00BF751D]
	GetHandleVerifier [0x00FF4513+2899763]
	GetHandleVerifier [0x0104793D+3240797]
	GetHandleVerifier [0x00DC13B4+593364]
	GetHandleVerifier [0x00DC82DC+621820]
	(No symbol) [0x00CD70A4]
	(No symbol) [0x00CD37A8]
	(No symbol) [0x00CD3947]
	(No symbol) [0x00CC59FE]
	BaseThreadInitThunk [0x761B7BA9+25]
	RtlInitializeExceptionChain [0x775ABE3B+107]
	RtlClearBits [0x775ABDBF+191]

Found 12 restaurants
Tên:Lẩu Bò Tí Chuột	Địa chỉ: 1 - 3 Cao Bá Nhạ, P. Nguyễn Cư Trinh	Quận